<a href="https://colab.research.google.com/github/rajashekarvt/Machine-Learning-Projects/blob/main/NLP_Using_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
unzip_data("nlp_getting_started.zip")

--2022-07-03 13:03:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-03 13:03:40 (64.0 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]

--2022-07-03 13:03:40--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 142.250.136.128, 142.250.148.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K)

In [14]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-520052eb-f2a8-09e5-5a78-0b2990372baa)


In [15]:
import pandas as pd
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [16]:
train_data_shuffled=train_data.sample(frac=1,random_state=42)
train_data_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [17]:
import random;
random_index=random.randint(0,len(train_data_shuffled)-5)
for row in train_data_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target=row
    print(f"Target:{target}","(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target:0 (not real disaster)
Text:
Do you feel engulfed with low self-image? Take the quiz: http://t.co/ykVsttvDWo http://t.co/IFQQpUr99X

---

Target:1 (real disaster)
Text:
I can probably skip on these basic life maintenance things for a few days. (cut to burning buildings people screaming in the streets)

---

Target:0 (not real disaster)
Text:
If you have an opinion and you don't put it on thh internet you will furst into flames.

---

Target:0 (not real disaster)
Text:
Hollywood Movie About Trapped Miners Released in Chile http://t.co/Fk1vyh5QLk #newsdict #news  #Chile

---

Target:1 (real disaster)
Text:
Inciweb OR Update:  Rogue River-Siskiyou National Forest Fires  8/5/15 12:00 PM (Rogue River-Siskiyou NF AreaÛ_ http://t.co/LkwxU8QV7n

---



In [18]:
from sklearn.model_selection import train_test_split;
x_train,x_test,y_train,y_test=train_test_split(train_data_shuffled["text"].to_numpy(),train_data_shuffled["target"].to_numpy(),test_size=0.25,random_state=42)

In [19]:
import tensorflow as tf;
from keras.layers import TextVectorization;
text_vectorizer=TextVectorization(max_tokens=10000,standardize="lower_and_strip_punctuation",split="whitespace",ngrams=None,
                                  output_mode="int",output_sequence_length=15)

TO SET THE MAX NO OF OUTPUT SEQUENCE LENGTH WE FIND THE AVERAGE OF THE TOKENS IN THE TRAIN TEXT SECTION WHICH NOW TURNED OUT OT BE 15
...AND ALSO MAX TOKENS CAN BE THE MAX ROWS THAT ARE IN UR TRAIN DATA OR IT CAN BE IN MULTIPLES OF THAT SAY I GOT 32K TEXT THEN I CHOOSE 10K MAX TOKENS EZ AS THAT!

In [20]:
round(sum([len(i.split()) for i in x_train])/len(x_train))

15

In [21]:
text_vectorizer.adapt(x_train)

In [22]:
#TRYING VECTORIZATION ON RANDOM SENTENCES
sentences=random.choice(x_train)
text_vectorizer([sentences])
print(f"ORIGINAL SENTENCE:\n{sentences}")
print(f"\nVECTORIZED SENTENCES:\n{text_vectorizer([sentences])}")

ORIGINAL SENTENCE:
When you're girlfriend is completely gorgeous???? @ woman crush &amp; stuff https://t.co/ycwAULQz3U

VECTORIZED SENTENCES:
[[  44  163 2612    9 1483 4864  410  518   34 1708    1    0    0    0
     0]]


In [23]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f"Total words in vocabulary:{len(words_in_vocab)}")
print(f"The Top 5 words in the vocabulary:{top_5_words}")
print(f"The bottom 5 words in the vocabulary:{bottom_5_words}")

Total words in vocabulary:10000
The Top 5 words in the vocabulary:['', '[UNK]', 'the', 'a', 'in']
The bottom 5 words in the vocabulary:['littledeath', 'littlebitofbass', 'litter\x89Ûª', 'lithium', 'literature']


In [24]:
tf.random.set_seed(42)
embedding=tf.keras.layers.Embedding(
    input_dim=10000,
    output_dim=128,
    embeddings_initializer="uniform",
    input_length=15,
    name="embedding_1"
)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer;
from sklearn.naive_bayes import MultinomialNB,GaussianNB;
from sklearn.pipeline import Pipeline;

model_0=Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])
model_0.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
baseline_score=model_0.score(x_test,y_test)
print(f"The accuracy using baseline model is {baseline_score*100:.2f}%")

The accuracy using baseline model is 79.41%


In [27]:
baseline_preds=model_0.predict(x_test)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support;
def calculate_results(y_test,y_pred):
    model_accuracy=accuracy_score(y_test,y_pred)*100
    model_precision,model_recall,model_fscore,_=precision_recall_fscore_support(y_test,y_pred,average="weighted")
    model_results={"accuracy":model_accuracy,
                   "precision":model_precision,
                   "recall":model_recall,
                   "f1-score":model_fscore}
    return model_results

In [29]:
baseline_results=calculate_results(y_test,baseline_preds)
print(baseline_results)

{'accuracy': 79.41176470588235, 'precision': 0.8114626597027842, 'recall': 0.7941176470588235, 'f1-score': 0.7870017995937115}


In [30]:
from keras import layers;
inputs=layers.Input(shape=(1,),dtype="string") #input is a string of 1d text
x=text_vectorizer(inputs)#turn the text into numbers!
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_dense")
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_1.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
179/179 [==============================] - 5s 6ms/step - loss: 0.6252 - accuracy: 0.6798 - val_loss: 0.5596 - val_accuracy: 0.7332
Epoch 2/5
179/179 [==============================] - 1s 5ms/step - loss: 0.4616 - accuracy: 0.8101 - val_loss: 0.4785 - val_accuracy: 0.7773
Epoch 3/5
179/179 [==============================] - 1s 5ms/step - loss: 0.3584 - accuracy: 0.8593 - val_loss: 0.4623 - val_accuracy: 0.7873
Epoch 4/5
179/179 [==============================] - 1s 5ms/step - loss: 0.2903 - accuracy: 0.8900 - val_loss: 0.4598 - val_accuracy: 0.7899
Epoch 5/5
179/179 [==============================] - 1s 6ms/step - loss: 0.2396 - accuracy: 0.9152 - val_loss: 0.4654 - val_accuracy: 0.7925


In [31]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [32]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.00141545,  0.01702217, -0.03451252, ..., -0.04689103,
         -0.01037428,  0.01904743],
        [ 0.04200266, -0.04297455, -0.03734074, ...,  0.00762704,
          0.03251547,  0.02850759],
        [ 0.01029128,  0.043913  ,  0.00205162, ..., -0.06045654,
         -0.04327291,  0.01042798],
        ...,
        [-0.09468728,  0.05706606, -0.09996226, ..., -0.04269457,
         -0.05467261,  0.08056024],
        [ 0.04933501,  0.01553482,  0.01593169, ..., -0.01781169,
          0.02554074, -0.00167966],
        [ 0.03103676, -0.00607105, -0.01014895, ...,  0.04555799,
         -0.02234115, -0.04768636]], dtype=float32)>]

In [33]:
embed_weights=model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [34]:

model_1_preds=model_1.predict(x_test)
model_1_preds[:10]

array([[0.38951105],
       [0.9285315 ],
       [0.99567324],
       [0.10732225],
       [0.08966835],
       [0.9320981 ],
       [0.9739446 ],
       [0.99031746],
       [0.9621451 ],
       [0.24797736]], dtype=float32)

In [35]:
model_1_pred=tf.squeeze(tf.round(model_1_preds))
model_1_results=calculate_results(y_test=y_test,y_pred=model_1_pred)
print(model_1_results)

{'accuracy': 79.25420168067227, 'precision': 0.7941317203460583, 'recall': 0.7925420168067226, 'f1-score': 0.7903389931821456}


In [36]:
import numpy as np;
np.array(list(baseline_results.values()))>np.array(list(model_1_results.values()))

array([ True,  True,  True, False])

In [37]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results, 
                                new_model_results=model_1_results)

Baseline accuracy: 79.41, New accuracy: 79.25, Difference: -0.16
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.00
Baseline f1-score: 0.79, New f1-score: 0.79, Difference: 0.00


************************** RECCURENT NEURAL NETWORKS (RNN) *******************************
One to one: one input, one output, such as image classification.
One to many: one input, many outputs, such as image captioning (image input, a sequence of text as caption output).
Many to one: many inputs, one outputs, such as text classification (classifying a Tweet as real diaster or not real diaster).
Many to many: many inputs, many outputs, such as machine translation (translating English to Spanish) or speech to text (audio wave as input, text as output).

When you come across RNN's in the wild, you'll most likely come across variants of the following:

Long short-term memory cells (LSTMs).
Gated recurrent units (GRUs).
Bidirectional RNN's (passes forward and backward along a sequence, left to right and right to left).

In [38]:
tf.random.set_seed(42)
from keras import layers;
model_2_embedding=layers.Embedding(input_dim=10000,output_dim=128,embeddings_initializer="uniform",input_length=15,name="embedding_2")
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=model_2_embedding(x)
x=layers.LSTM(64)(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_LSTM")
model_2.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model_2.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
179/179 [==============================] - 6s 11ms/step - loss: 0.5237 - accuracy: 0.7355 - val_loss: 0.4554 - val_accuracy: 0.7899
Epoch 2/5
179/179 [==============================] - 1s 8ms/step - loss: 0.3063 - accuracy: 0.8767 - val_loss: 0.4793 - val_accuracy: 0.7721
Epoch 3/5
179/179 [==============================] - 1s 8ms/step - loss: 0.1983 - accuracy: 0.9319 - val_loss: 0.7552 - val_accuracy: 0.7489
Epoch 4/5
179/179 [==============================] - 1s 7ms/step - loss: 0.1419 - accuracy: 0.9513 - val_loss: 0.6684 - val_accuracy: 0.7532
Epoch 5/5
179/179 [==============================] - 1s 7ms/step - loss: 0.1086 - accuracy: 0.9641 - val_loss: 0.6667 - val_accuracy: 0.7463


In [39]:
model_2_preds=model_2.predict(x_test)
model_2_pred=tf.squeeze(tf.round(model_2_preds))
model_2_results=calculate_results(y_test,model_2_pred)
print(model_2_results)
np.array(list(baseline_results.values()))<np.array(list(model_2_results.values()))

{'accuracy': 74.63235294117648, 'precision': 0.7480973492714685, 'recall': 0.7463235294117647, 'f1-score': 0.7468299194995421}


array([False, False, False, False])

In [40]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [41]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.41, New accuracy: 74.63, Difference: -4.78
Baseline precision: 0.81, New precision: 0.75, Difference: -0.06
Baseline recall: 0.79, New recall: 0.75, Difference: -0.05
Baseline f1-score: 0.79, New f1-score: 0.75, Difference: -0.04


In [42]:
tf.random.set_seed(42)
model_3_embedding=tf.keras.layers.Embedding(input_dim=10000,output_dim=128,embeddings_initializer="uniform",
                                            input_length=15,name="embedding_3")
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=model_3_embedding(x)
x=layers.GRU(64)(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_GRU")
model_3.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model_3.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
179/179 [==============================] - 5s 17ms/step - loss: 0.5416 - accuracy: 0.7159 - val_loss: 0.4552 - val_accuracy: 0.7967
Epoch 2/5
179/179 [==============================] - 1s 7ms/step - loss: 0.3125 - accuracy: 0.8727 - val_loss: 0.4700 - val_accuracy: 0.7820
Epoch 3/5
179/179 [==============================] - 1s 7ms/step - loss: 0.1965 - accuracy: 0.9299 - val_loss: 0.7141 - val_accuracy: 0.7442
Epoch 4/5
179/179 [==============================] - 1s 7ms/step - loss: 0.1388 - accuracy: 0.9520 - val_loss: 0.6491 - val_accuracy: 0.7516
Epoch 5/5
179/179 [==============================] - 1s 7ms/step - loss: 0.1122 - accuracy: 0.9627 - val_loss: 0.6092 - val_accuracy: 0.7516


In [43]:
model_3_preds=model_3.predict(x_test)
model_3_pred=tf.squeeze(tf.round(model_3_preds))
model_3_results=calculate_results(y_test,model_3_pred)
print(model_3_results)

{'accuracy': 75.15756302521008, 'precision': 0.7513356420247913, 'recall': 0.7515756302521008, 'f1-score': 0.7514387531122902}


In [44]:
np.array(list(baseline_results.values()))<np.array(list(model_3_results.values()))

array([False, False, False, False])

In [45]:
compare_baseline_to_new_results(baseline_results,model_3_results)

Baseline accuracy: 79.41, New accuracy: 75.16, Difference: -4.25
Baseline precision: 0.81, New precision: 0.75, Difference: -0.06
Baseline recall: 0.79, New recall: 0.75, Difference: -0.04
Baseline f1-score: 0.79, New f1-score: 0.75, Difference: -0.04


In [46]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [47]:
tf.random.set_seed(42)
model_4_embedding=tf.keras.layers.Embedding(input_dim=10000,output_dim=128,embeddings_initializer="uniform",
                                            input_length=15,name="embedding_4")
inputs=tf.keras.layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=model_4_embedding(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
outputs=tf.keras.layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")
model_4.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model_4.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
179/179 [==============================] - 6s 13ms/step - loss: 0.5198 - accuracy: 0.7395 - val_loss: 0.4576 - val_accuracy: 0.7915
Epoch 2/5
179/179 [==============================] - 2s 9ms/step - loss: 0.3015 - accuracy: 0.8779 - val_loss: 0.4870 - val_accuracy: 0.7794
Epoch 3/5
179/179 [==============================] - 2s 10ms/step - loss: 0.1886 - accuracy: 0.9341 - val_loss: 0.6402 - val_accuracy: 0.7505
Epoch 4/5
179/179 [==============================] - 2s 11ms/step - loss: 0.1293 - accuracy: 0.9562 - val_loss: 0.7331 - val_accuracy: 0.7558
Epoch 5/5
179/179 [==============================] - 3s 15ms/step - loss: 0.0995 - accuracy: 0.9658 - val_loss: 0.7238 - val_accuracy: 0.7500


In [48]:
model_4_preds=model_4.predict(x_test)
model_4_pred=tf.squeeze(tf.round(model_4_preds))
model_4_results=calculate_results(y_test,model_4_pred)
print(model_4_results)

{'accuracy': 75.0, 'precision': 0.7508602456479239, 'recall': 0.75, 'f1-score': 0.7503108098322853}


In [49]:
np.array(list(baseline_results.values()))<np.array(list(model_4_results.values()))

array([False, False, False, False])

In [50]:
compare_baseline_to_new_results(baseline_results,model_4_results)

Baseline accuracy: 79.41, New accuracy: 75.00, Difference: -4.41
Baseline precision: 0.81, New precision: 0.75, Difference: -0.06
Baseline recall: 0.79, New recall: 0.75, Difference: -0.04
Baseline f1-score: 0.79, New f1-score: 0.75, Difference: -0.04


In [51]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [52]:
tf.random.set_seed(42)
model_5_embedding=tf.keras.layers.Embedding(input_dim=10000,output_dim=128,embeddings_initializer="uniform",
                                            input_length=15,name="embedding_5")
inputs=tf.keras.layers.Input((1,),dtype="string")
x=text_vectorizer(inputs)
x=model_5_embedding(x)
x=layers.Conv1D(filters=32,kernel_size=5,activation="relu")(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_5=tf.keras.Model(inputs,outputs,name="model_5_conv1D")
model_5.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
model_5.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
179/179 [==============================] - 9s 7ms/step - loss: 0.5517 - accuracy: 0.7176 - val_loss: 0.4750 - val_accuracy: 0.7915
Epoch 2/5
179/179 [==============================] - 1s 5ms/step - loss: 0.3300 - accuracy: 0.8630 - val_loss: 0.5035 - val_accuracy: 0.7773
Epoch 3/5
179/179 [==============================] - 1s 5ms/step - loss: 0.2172 - accuracy: 0.9168 - val_loss: 0.5822 - val_accuracy: 0.7684
Epoch 4/5
179/179 [==============================] - 1s 5ms/step - loss: 0.1472 - accuracy: 0.9450 - val_loss: 0.6873 - val_accuracy: 0.7668
Epoch 5/5
179/179 [==============================] - 1s 5ms/step - loss: 0.1102 - accuracy: 0.9599 - val_loss: 0.7591 - val_accuracy: 0.7631


In [53]:
model_5_preds=model_5.predict(x_test)
model_5_pred=tf.squeeze(tf.round(model_5_preds))
model_5_results=calculate_results(y_test,model_5_pred)
print(model_5_results)

{'accuracy': 76.31302521008404, 'precision': 0.7626576062612739, 'recall': 0.7631302521008403, 'f1-score': 0.761749908851685}


In [54]:
compare_baseline_to_new_results(baseline_results,model_5_results)

Baseline accuracy: 79.41, New accuracy: 76.31, Difference: -3.10
Baseline precision: 0.81, New precision: 0.76, Difference: -0.05
Baseline recall: 0.79, New recall: 0.76, Difference: -0.03
Baseline f1-score: 0.79, New f1-score: 0.76, Difference: -0.03


In [55]:
np.array(list(baseline_results.values()))<np.array(list(model_5_results.values()))

array([False, False, False, False])

In [56]:
import tensorflow_hub as hub;
embed=hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [58]:
sentence_encoding_layer=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                       input_shape=[],
                                       trainable=False,
                                       dtype=tf.string,
                                       name="USE")
model_6=tf.keras.models.Sequential([
    sentence_encoding_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name="model_6_USE")

model_6.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_6.fit(x_train,y_train,epochs=5,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/5
179/179 [==============================] - 6s 16ms/step - loss: 0.5118 - accuracy: 0.7888 - val_loss: 0.4524 - val_accuracy: 0.7957
Epoch 2/5
179/179 [==============================] - 2s 14ms/step - loss: 0.4125 - accuracy: 0.8122 - val_loss: 0.4416 - val_accuracy: 0.7988
Epoch 3/5
179/179 [==============================] - 2s 14ms/step - loss: 0.3967 - accuracy: 0.8247 - val_loss: 0.4455 - val_accuracy: 0.8004
Epoch 4/5
179/179 [==============================] - 2s 14ms/step - loss: 0.3885 - accuracy: 0.8273 - val_loss: 0.4412 - val_accuracy: 0.7988
Epoch 5/5
179/179 [==============================] - 2s 14ms/step - loss: 0.3806 - accuracy: 0.8334 - val_loss: 0.4395 - val_accuracy: 0.8025


In [71]:
model_6_preds=model_6.predict(x_test)
model_6_pred=tf.squeeze(tf.round(model_6_preds))
model_6_results=calculate_results(y_test,model_6_pred)
print(model_6_results)

{'accuracy': 80.25210084033614, 'precision': 0.8029041642996092, 'recall': 0.8025210084033614, 'f1-score': 0.8011852736321583}


In [73]:
compare_baseline_to_new_results(baseline_results,model_6_results)

Baseline accuracy: 79.41, New accuracy: 80.25, Difference: 0.84
Baseline precision: 0.81, New precision: 0.80, Difference: -0.01
Baseline recall: 0.79, New recall: 0.80, Difference: 0.01
Baseline f1-score: 0.79, New f1-score: 0.80, Difference: 0.01


In [67]:
np.array(list(baseline_results.values()))<np.array(list(model_6_results.values()))

array([ True, False,  True,  True])

In [76]:
train_data_90_percent,train_data_10_percent,y_train_90_percent,y_train_10_percent=train_test_split(x_train,y_train,test_size=0.1,random_state=42)

In [74]:
model_7=tf.keras.models.clone_model(model_6)

In [78]:
model_7.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model_7.fit(train_data_10_percent,y_train_10_percent,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
18/18 [==============================] - 5s 134ms/step - loss: 0.4302 - accuracy: 0.8371 - val_loss: 0.4886 - val_accuracy: 0.7768
Epoch 2/5
18/18 [==============================] - 1s 81ms/step - loss: 0.4002 - accuracy: 0.8406 - val_loss: 0.4826 - val_accuracy: 0.7679
Epoch 3/5
18/18 [==============================] - 1s 78ms/step - loss: 0.3794 - accuracy: 0.8494 - val_loss: 0.4792 - val_accuracy: 0.7742
Epoch 4/5
18/18 [==============================] - 2s 91ms/step - loss: 0.3614 - accuracy: 0.8511 - val_loss: 0.4799 - val_accuracy: 0.7710
Epoch 5/5
18/18 [==============================] - 1s 79ms/step - loss: 0.3459 - accuracy: 0.8599 - val_loss: 0.4813 - val_accuracy: 0.7715


In [80]:
model_7_preds=model_7.predict(x_test)
model_7_pred=tf.squeeze(tf.round(model_7_preds))
model_7_results=calculate_results(y_test,model_7_pred)
print(model_7_results)

{'accuracy': 77.15336134453781, 'precision': 0.7714559027357701, 'recall': 0.7715336134453782, 'f1-score': 0.7714928323670225}


In [83]:
all_model_results = pd.DataFrame({"baseline": baseline_results,
                                  "simple_dense": model_1_results,
                                  "lstm": model_2_results,
                                  "gru": model_3_results,
                                  "bidirectional": model_4_results,
                                  "conv1d": model_5_results,
                                  "tf_hub_sentence_encoder": model_6_results,
                                  "tf_hub_10_percent_data": model_7_results})
all_model_results = all_model_results.T
all_model_results

,accuracy,precision,recall,f1-score
baseline,79.411765,0.811463,0.794118,0.787002
simple_dense,79.254202,0.794132,0.792542,0.790339
lstm,74.632353,0.748097,0.746324,0.746830
gru,75.157563,0.751336,0.751576,0.751439
bidirectional,75.000000,0.750860,0.750000,0.750311
conv1d,76.313025,0.762658,0.763130,0.761750
tf_hub_sentence_encoder,80.252101,0.802904,0.802521,0.801185
tf_hub_10_percent_data,77.153361,0.771456,0.771534,0.771493


In [94]:
model_6.predict(["hi my name is rajashekar and this is not supposed to be a disaster! it is very dangerous and mindblowing bomb devestation bombed terrorist "])

array([[0.9057223]], dtype=float32)